In [ ]:
# pip install xlrd
# pip install pandas

In [1]:
# Subindo os dados de descargas de cada UN
import pandas as pd
import os

Descargas_EPB = pd.read_csv(f'/home/ruanv/DESCARGAS_2023/Consolidado Raios 2023_EPB.csv', delimiter=';')
Descargas_ERO = pd.read_csv(f'/home/ruanv/DESCARGAS_2023/Consolidado Raios 2023_ERO.csv', delimiter=';')
Descargas_EMT = pd.read_csv(f'/home/ruanv/DESCARGAS_2023/Consolidado Raios 2023_EMT.csv', delimiter=';')
Descargas_EMS = pd.read_csv(f'/home/ruanv/DESCARGAS_2023/Consolidado Raios 2023_EMS.csv', delimiter=';')
Descargas_EMR = pd.read_csv(f'/home/ruanv/DESCARGAS_2023/Consolidado Raios 2023_EMR.csv', delimiter=';')
Descargas_EAC = pd.read_csv(f'/home/ruanv/DESCARGAS_2023/Consolidado Raios 2023_EAC.csv', delimiter=';')
Descargas_ETO = pd.read_csv(f'/home/ruanv/DESCARGAS_2023/Consolidado Raios 2023_ETO.csv', delimiter=';')
Descargas_ESS = pd.read_csv(f'/home/ruanv/DESCARGAS_2023/Consolidado Raios 2023_ESS.csv', delimiter=';')
Descargas_ESE = pd.read_csv(f'/home/ruanv/DESCARGAS_2023/Consolidado Raios 2023_ESE.csv', delimiter=';')

# Deixando um único dataframe pra todas as descargas do grupo Energisa
Lista_Descargas = [Descargas_EPB, Descargas_ERO, Descargas_EMT, Descargas_EMS, Descargas_EMR, Descargas_EAC, Descargas_ETO, Descargas_ESS, Descargas_ESE]
Descargas_Energisa = pd.concat(Lista_Descargas, ignore_index=True)

# Mantendo apenas as colunas da base que possuem dados de raios no sentido Nuvem-Solo
Descargas_Energisa = Descargas_Energisa.iloc[:, [0, 1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 34]]

#  Excluir a primeira linha, pois a tabela só começa na terceira linha
Descargas_Energisa = Descargas_Energisa.drop(0)

# Renomear todas as colunas para que fique de acordo com o número de raios nos meses correspondentes 
novos_nomes_colunas = ['Localidade', 'Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']
Descargas_Energisa.columns = novos_nomes_colunas


In [2]:
# Carregando as bases de Ocorrências da ANEEL e dos códigos do IBGE
Cod_IBGE = pd.read_excel(f'/home/ruanv/Documentos/cod_ibge.xls', header=6)
Ocorrencias = pd.read_csv(f'/home/ruanv/Documentos/ocorrencias-emergenciais-rede-distribuicao-2023.csv', delimiter=';', encoding='latin-1', low_memory=False)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'


In [ ]:
# Adicionando a coluna de código IBGE no dataframe com os dados de descargas atmosféricas

In [4]:
print(Ocorrencias.columns)

Index(['DatGeracaoConjuntoDados', 'NomAgente', 'NumCPFCNPJ', 'NumOcorrencia',
       'NumConjunto', 'DthInicioOcorrenciaAberta', 'DscCanalAtendimento',
       'DthFimOcorrenciaAberta', 'DscOcorrenciaAberta', 'DscNumInterrupcao',
       'MdaPreparo', 'MdaDeslocamento', 'MdaExecucao', 'NumVeiculo',
       'CodIBGE'],
      dtype='object')


In [ ]:
# Filtrando apenas ocorrências que sejam advindas de descargas atmosféricas
Ocorrencias = Ocorrencias[Ocorrencias['DscOcorrenciaAberta'] == "INTERNA;NAO PROGRAMADA;MEIO AMBIENTE;DESCARGA ATMOSFERICA"]


In [ ]:
# Mesclando a tabela de Ocorrências com o nome dos municípios, conforme código IBGE
Ocorrencias = pd.merge(Ocorrencias, Cod_IBGE, left_on='CodIBGE', right_on='Código Município Completo', how='left')

In [ ]:
print(Cod_IBGE)

In [ ]:
print(Ocorrencias)

In [ ]:
print(Descargas_Energisa)

In [ ]:
import pandas as pd

# Supondo que você tenha um DataFrame chamado Cod_IBGE com as colunas "Município" e "Nome_UF"

# Supondo que você já tenha os DataFrames Descargas_EPB, Descargas_ERO, etc.
# Substitua isso pelos seus DataFrames reais

# Agora, adicionando a coluna "Cod_IBGE" para cada DataFrame na Lista_Descargas
for base in Lista_Descargas:
    # Obtém o estado da base atual
    estado = base.iloc[0]['Localidade']
    
    # Filtra o DataFrame Cod_IBGE para o estado correspondente
    cod_ibge_estado = Cod_IBGE[Cod_IBGE['Nome_UF'] == estado]
    
    # Mescla os DataFrames com base no município
    base = pd.merge(base, cod_ibge_estado[['Nome_Município', 'Cod_IBGE']], how='left', left_on='Localidade', right_on='Município')
    
    # Remove a coluna redundante
    base = base.drop(columns='Município')
    
    # Renomeia a coluna Cod_IBGE
    base = base.rename(columns={'Cod_IBGE': 'Cod_IBGE'})

# Agora, cada DataFrame na Lista_Descargas terá uma nova coluna 'Cod_IBGE'
